<a href="https://colab.research.google.com/github/SarkarPriyanshu/OasisInfobyteTasks/blob/main/Oasis_Infobyte_Internships.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Oasis Infobyte Task 1

In [1]:
! pip install -q plotly
! pip install -q dash
! pip install -q jupyter-dash

! pip install -q kaggle

! chmod 600 ~/.kaggle/kaggle.json

! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

!kaggle datasets download -d gokulrajkmv/unemployment-in-india

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.7 MB/s eta 0:00:00
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
  0% 0.00/16.0k [00:00<?, ?B/s]
100% 16.0k/16.0k [00:00<00:00, 29.5MB/s]


In [2]:
import zipfile
zip_ref = zipfile.ZipFile("/content/unemployment-in-india.zip", 'r')
zip_ref.extractall("/content/")
zip_ref.close()

In [118]:
import pandas as pd
import numpy as np
import plotly.express as px

from dash import Dash,dcc,html
from dash.exceptions import PreventUpdate
from dash.dependencies import Input,Output
from jupyter_dash import JupyterDash

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [217]:
df1 =pd.read_csv('/content/Unemployment in India.csv')
df2 =pd.read_csv('/content/Unemployment_Rate_upto_11_2020.csv')

df1.rename(columns=lambda x:x.strip(),inplace=True)
df2.rename(columns=lambda x:x.strip(),inplace=True)

- **Region**: This column tells us the specific area or location where the data was collected. It could be a city, a state, or even a country, depending on the scope of your dataset.

- **Estimated Employed**: This column shows the number of people who are currently employed in the specified region. It's an estimate because it's not always possible to count every single person, so statisticians use methods to make an educated guess.

- **Estimated Labour Participation Rate (%)**: This column represents the percentage of people in the region who are either employed or actively looking for work. So, it's like a measure of how many people are either working or trying to find a job compared to the total population of working-age individuals.

- **Date**: This column indicates the date when the data was collected or recorded. It helps us track changes over time and see how employment and unemployment rates evolve during the COVID-19 pandemic.

- **Estimated Unemployment Rate (%)**: This column shows the percentage of people in the region who are actively looking for a job but currently do not have one. It's a measure of joblessness in the area.

- **Part**: This column might refer to a specific part or division within the region, providing more detailed information about where the data was collected. For example, it could be a district within a city or a county within a state.

- **Month**: This column likely represents the month associated with the data. It helps organize the information chronologically, allowing us to see trends and patterns month by month.


In [5]:
common_columns = list(set(df1.columns).intersection(set(df2.columns)))

In [218]:
merge_df = pd.concat((df1[common_columns],df2[common_columns]),axis=0)

In [86]:
merge_df.sample(5)

,Frequency,Region,Estimated Employed,Estimated Labour Participation Rate (%),Date,Estimated Unemployment Rate (%),Part,Month,Year
839,Monthly,Jharkhand,10198029.0,42.92,2020-01-31,10.61,East,January,2020
366,Monthly,Andhra Pradesh,4913963.0,38.61,2019-12-31,7.88,South,December,2019
718,Monthly,Uttarakhand,904903.0,33.61,2019-11-30,6.34,North,November,2019
705,Monthly,Uttar Pradesh,13857200.0,40.21,2019-12-31,12.37,North,December,2019
622,Monthly,Punjab,3602243.0,41.82,2019-11-30,10.39,North,November,2019


In [8]:
merge_df.shape

(1035, 6)

In [9]:
np.round(merge_df.isna().sum() / merge_df.shape[0] * 100,2)

Frequency                                  2.71
Region                                     2.71
Estimated Employed                         2.71
Estimated Labour Participation Rate (%)    2.71
Date                                       2.71
Estimated Unemployment Rate (%)            2.71
dtype: float64

In [220]:
merge_df.dropna(inplace=True)

In [221]:
merge_df.reset_index(drop=True,inplace=True)

In [222]:
merge_df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Frequency,1007,3,Monthly,381,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Region,1007,28,Andhra Pradesh,38,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Estimated Employed,1007.0,NaN,NaN,NaN,8996209.182721,10207838.810233,49420.0,1639125.0,5543380.0,12871154.0,59433759.0
Estimated Labour Participation Rate (%),1007.0,NaN,NaN,NaN,42.37862,8.048542,13.33,37.835,40.88,45.315,72.57
Date,1007,18,29-02-2020,79,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Estimated Unemployment Rate (%),1007.0,NaN,NaN,NaN,11.906991,10.739549,0.0,4.685,8.89,16.125,76.74


In [224]:
merge_df = merge_df.astype({'Date':'datetime64[ns]'})

<ipython-input-224-dd687151bf6a>:1: UserWarning:

Parsing dates in  %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.



In [225]:
Parts = {value[0]:value[1] for value in df2.groupby(['Region','Region.1'],as_index=False).count().iloc[:,:2].values}

merge_df = merge_df.assign(
    Part = ['North' if value == 'Chandigarh' else Parts[value] for value in merge_df['Region']],
    Month = merge_df['Date'].dt.month_name(),
    Year = merge_df['Date'].dt.year
)

In [42]:
merge_df.sample(5)

,Frequency,Region,Estimated Employed,Estimated Labour Participation Rate (%),Date,Estimated Unemployment Rate (%),Part,Month
351,Monthly,West Bengal,25630359.0,50.00,2019-11-30,5.83,East,November
428,Monthly,Delhi,5550172.0,42.71,2019-06-30,12.76,North,June
902,Monthly,Odisha,6865693.0,25.23,2020-04-30,23.76,East,April
294,Monthly,Telangana,12636415.0,72.57,2019-09-30,5.49,South,September
668,Monthly,Tamil Nadu,5733921.0,23.77,2020-05-31,25.95,South,May


In [226]:
merge_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1007 entries, 0 to 1006
Data columns (total 9 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   Frequency                                1007 non-null   object        
 1   Region                                   1007 non-null   object        
 2   Estimated Employed                       1007 non-null   float64       
 3   Estimated Labour Participation Rate (%)  1007 non-null   float64       
 4   Date                                     1007 non-null   datetime64[ns]
 5   Estimated Unemployment Rate (%)          1007 non-null   float64       
 6   Part                                     1007 non-null   object        
 7   Month                                    1007 non-null   object        
 8   Year                                     1007 non-null   int32         
dtypes: datetime64[ns](1), float64(3), int32(1)

### Data Analysis

In [47]:
fig = px.histogram(merge_df,
                   x="Estimated Unemployment Rate (%)",
                   color="Part",
                   marginal="box",
                   title='Estimated Unemployment Rate (%) in different parts of India'
                   )
fig.show()

- The South and Northeast regions generally have lower median unemployment rates compared to the East, West, and North regions. This suggests that, on average, the South and Northeast have better employment conditions. But over time there is certain peak point where south states struggle in terms of unemployment we can clearly see that in line chart during MAY 2020 - JULY 2020.

- However, the South has the widest dispersion, indicating greater variability in unemployment rates across its areas. This could imply more significant disparities in employment opportunities or economic conditions within the region.

- On the other hand, the East has the highest median unemployment rate among the regions, indicating that, on average, it may have more significant unemployment challenges compared to the others.

- The West region generally has lower median unemployment rates compared to the other regions, suggesting that, on average, it may have better employment conditions.

#### Are there any noticeable trends or patterns in the Estimated Unemployment Rate (%) across regions over the months?

In [68]:
avgLabourParticipationRate = merge_df.groupby(['Date','Part'],as_index=False).agg({'Estimated Unemployment Rate (%)':'mean'})
fig = px.line(avgLabourParticipationRate,
              x='Date',
              y='Estimated Unemployment Rate (%)',
              color='Part',
              markers=True,
              title='Average Estimated Unemployment Rate (%) over time.')
fig.show()

In [46]:
fig = px.histogram(merge_df,
                   x="Estimated Labour Participation Rate (%)",
                   color="Part",
                   marginal="box",
                   title='Estimated Labour Participation Rate (%) in different parts of India'
                   )
fig.show()

- The Northeast region generally has higher median participation rates compared to other regions.
- The South region has the widest dispersion, indicating more variability in participation rates across different areas. This could suggest disparities in employment opportunities or economic conditions within the region.
- The East and North regions appear to have more consistent participation rates, with narrower dispersions and median values around 40%.

#### How has the Estimated Employed changed over time across different regions during the COVID-19 pandemic?

In [70]:
avgLabourParticipationRate = merge_df.groupby(['Date','Part'],as_index=False).agg({'Estimated Labour Participation Rate (%)':'mean'})
fig = px.line(avgLabourParticipationRate,
              x='Date',
              y='Estimated Labour Participation Rate (%)',
              color='Part',
              markers=True,
              title='Estimated Labour Participation Rate (%) over time.'
              )
fig.show()

#### What is the relationship between unemployment and labor participation

In [54]:
fig = px.scatter(merge_df, x="Estimated Labour Participation Rate (%)",
                 y="Estimated Unemployment Rate (%)",
                 color="Part",
                 size='Estimated Unemployment Rate (%)')
fig.show()

- As the What unemployment rate increases and labor participation it results in lower the Labour Participation.
- We saw this in line chart above as the during high peak in May 2020 - july 2020 there was a drop in Labour Participation in different part of country.

#### Which region experienced the highest and lowest Estimated Labour Participation Rate (%) on average during the pandemic?
- South and Northeast states had highly affected by Pandamic because of higher average Estimated Labour Participation Rate and Estimated Unemployment Rate.
- People were more desperately seeking for new oppotunities in South and Northeast states.
- East is second affected part of country that shows Unemployment Rate.
- West and North least affected by Pandamic.

#### Which state Estimated Unemployment Rate highly and least affected during the COVID-19 pandemic?

In [238]:
fig = px.bar(merge_df, x='Region', y='Estimated Unemployment Rate (%)', animation_frame='Month',
             title='Unemployment rate across regions', height=700, template='seaborn')


fig.update_layout(xaxis={'categoryorder': 'total descending'})

fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1000
fig.show()

In [138]:
fig = px.box(merge_df,
             x='Region',
             y='Estimated Unemployment Rate (%)',
             color='Region',
             notched=True,
             hover_data=["Part"],
             title='Unemployment rate per States', template='seaborn')

# Updating the x-axis category order to be in descending total
fig.update_layout(xaxis={'categoryorder': 'total descending'})
fig.show()

- Puducherry have a sudden spike in 2020 april.

#### Top Regions in terms of Unemployment Rate.

In [241]:
averageUnemploymentRegions = merge_df\
                              .groupby('Region',as_index=False)\
                              .agg({'Estimated Unemployment Rate (%)':'mean'})\
                              .nlargest(5,'Estimated Unemployment Rate (%)')

fig = px.bar(averageUnemploymentRegions, x="Region", y="Estimated Unemployment Rate (%)",title='Highly affected states during Pandamic.')

fig.show()


In [229]:
topValues = list(averageUnemploymentRegions['Region'].values)
topValues = merge_df.query('Region in @topValues').groupby(['Region','Date']).agg({'Estimated Labour Participation Rate (%)':'mean'}).reset_index().dropna()
fig = px.line(topValues,
              x='Date',
              y='Estimated Labour Participation Rate (%)',
              color='Region',
              markers=True,
              title='Estimated Labour Participation Rate (%) over time for top 5 Regions.'
              )
fig.show()

#### Bottom Regions in terms of Unemployment Rate.

In [234]:
averageUnemploymentRegions = merge_df\
                              .groupby('Region',as_index=False)\
                              .agg({'Estimated Unemployment Rate (%)':'mean'})\
                              .nsmallest(5,'Estimated Unemployment Rate (%)')

fig = px.bar(averageUnemploymentRegions, x="Region", y="Estimated Unemployment Rate (%)",title='Least affected states during pandamic')

fig.show()


In [239]:
topValues = list(averageUnemploymentRegions['Region'].values)
topValues = merge_df.query('Region in @topValues').groupby(['Region','Date']).agg({'Estimated Labour Participation Rate (%)':'mean'}).reset_index().dropna()
fig = px.line(topValues,
              x='Date',
              y='Estimated Labour Participation Rate (%)',
              color='Region',
              markers=True,
              title='Estimated Labour Participation Rate (%) over time for bottom 5 Regions.'
              )
fig.show()

#### Which Part or State affected by Unemployment rate?

In [245]:
unemployed = merge_df.groupby(['Part', 'Region'])['Estimated Unemployment Rate (%)'].mean().reset_index()

# Creating a Sunburst chart
fig = px.sunburst(unemployed, path=['Part', 'Region'], values='Estimated Unemployment Rate (%)', color_continuous_scale='rdylbu',
                  title='Unemployment rate in each Part and State', height=550, template='presentation')

fig.show()

- As we saw in previous observation South and Northeast states are affected by Unemployment
- States from North part of India like Delhi, Haryana, where Bihar and Jharkhand in east and Tipura in Northeast and kerala in south got affected.

In [19]:
regions_list = list(merge_df['Region'].unique())
external_stylesheets = ['https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css']
app = Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.H3('How different regions got affected in terms of employement over time?'),
    html.Br(),
    html.Div([
      html.Br(),
      html.Label('Select a region'),
      dcc.Dropdown(
            options = regions_list,
            id='select-region'
        ),
    html.Div(id='statistics',className='container'),
    dcc.Graph(id='line-plot')
    ])
])

@app.callback([Output('line-plot','figure' ),
                Output('statistics', 'children')],
              [Input('select-region','value')])


def updateOutput(selected_region):
  if not selected_region:
    raise PreventUpdate

  filtered_data = merge_df\
                .query(f'Region == "{selected_region}"')\
                .sort_values(by='Date')[['Region','Date','Estimated Unemployment Rate (%)']]

  unemplyment_average = filtered_data['Estimated Unemployment Rate (%)'].mean()
  unemplyment_median = filtered_data['Estimated Unemployment Rate (%)'].median()
  unemplyment_std = filtered_data['Estimated Unemployment Rate (%)'].std()

  statistics_text = html.Div([

        html.Br(),
        html.Div(className='col-12 d-flex justify-content-around align-items-center' ,children=[
                              html.P(className="text-primary text-uppercase",children=f"Mean: {unemplyment_average:.2f}\t\t"),
                              html.P(className="text-primary text-uppercase",children=f"Median: {unemplyment_median:.2f}\t\t"),
                              html.P(className="text-primary text-uppercase",children=f"Standard Deviation: {unemplyment_std:.2f}\t\t")
                              ]),
        html.Div(className='row d-flex justify-content-between', children=[
                    html.Div(className='col-6 justify-content-center d-flex align-items-center',children=[
                              dcc.Graph(id='hist-plot',
                                        figure=px.histogram(filtered_data,
                                                            x="Estimated Unemployment Rate (%)",
                                                             title=f'Estimated Unemployment distribution in {selected_region}.'
                                                             )
                                        )
                              ]),
                    html.Div(className='col-6 justify-content-center d-flex align-items-center',children=[
                              dcc.Graph(id='hist-plot',
                                        figure=px.box(filtered_data,
                                                            x="Estimated Unemployment Rate (%)",
                                                             title=f'Estimated Unemployment boxplot in {selected_region}.'
                                                             )
                                    )
                          ])
                  ])

      ])

  fig =  px.line(filtered_data, x="Date", y="Estimated Unemployment Rate (%)", title=f'Estimated Unemployment in {selected_region} over time.')
  return fig,statistics_text

if __name__ == "__main__":
  app.run_server(debug=True,mode='inline')

<IPython.core.display.Javascript object>

#### Impact of Lockdown on States Estimated Employed

In [248]:
df2 = df2.astype({'Date':'datetime64[ns]'})

df2['Month'] = df2['Date'].dt.month_name()

In [251]:
fig = px.scatter_geo(df2,'longitude', 'latitude', color="Region",
                     hover_name="Region", size="Estimated Unemployment Rate (%)",
                     animation_frame="Month",scope='asia',template='seaborn',title='Impack of lockdown on Employement across regions')

fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 3000

fig.update_geos(lataxis_range=[5,35], lonaxis_range=[65, 100],oceancolor="#3399FF",
    showocean=True)

fig.show()